# PySESync: SE-Sync Python bindings 🐍

This notebook demonstrates the use of SE-Sync's Python bindings

In [ ]:
import sys
import numpy as np
from matplotlib import pyplot as plt

Import the SE-Sync Python library built using pybind

In [ ]:
sesync_lib_path = "../build/lib"  # Relative path to libraries built by the SE-Sync CMake project
sys.path.insert(0, sesync_lib_path)  # Add this directory to Python interpreter path

In [ ]:
import PySESync

Load in some data from a .g2o file

In [ ]:
filename = "../../data/intel.g2o"

measurements, num_poses = PySESync.read_g2o_file(filename)

d = measurements[0].R.shape[0]

print("Loaded %d measurements between %d %d-dimensional poses from file %s" % (len(measurements), num_poses, d, filename))

**Run SE-Sync!**

Set SE-Sync options

In [ ]:
opts = PySESync.SESyncOpts()
opts.num_threads = 4
opts.verbose=True

opts.r0 = d  # Start at lowest possible level of the Riemannian Staircase
opts.formulation = PySESync.Formulation.Explicit    # Options are: Simplified, Explicit, SOSync (rotation averaging)
opts.initialization = PySESync.Initialization.Random   # Options are Chordal or Random

In [ ]:
result = PySESync.SESync(measurements, opts)

**Plot some interesting data about the solution process**

In [ ]:
# Plot sequence of Riemannian gradient norms
grad_norms = np.concatenate(result.gradient_norms)
plt.semilogy(grad_norms)
plt.title("Norm of Riemannian gradient")
plt.xlabel("Iteration")
plt.show()

In [ ]:
# Plot sequence of objective values
fvals = np.concatenate(result.function_values)
plt.semilogy(fvals)
plt.title("Objective value")
plt.xlabel("Iteration")
plt.show()

**Plot estimated SLAM solution**

In [ ]:
# Extract translational states from solution xhat
xhat = result.xhat
R0inv = np.linalg.inv(xhat[:, num_poses : num_poses + 2])
t = np.matmul(R0inv, xhat[:, 0:num_poses])

# Plot the position of each estimated pose, together with the edges between them
for k in range(0, len(measurements)):
    plt.plot(t[0, [measurements[k].i, measurements[k].j]], t[1, [measurements[k].i, measurements[k].j]], '.-b')

plt.title("Estimated SLAM solution")

plt.gca().set_aspect('equal')
plt.show()